# **Problem 1: Wastewater Problem**

In [2]:
# To solve this simulation problem, we will import the Numpy library (numpy is short for 'Numerical Python')
import numpy as np

# Constant variables as defined in the problem statement
condos = 143 # number of condos in the resort complex
occupancy_rate = 0.87 # as defined in the problem statement
mean_wastewater = 22.3 # in gallons
standard_deviation_wastewater = 19.2 # in gallons
treatment_device_expense = 33.4 # in gallons
treatment_capacity = 500 # in gallons
pollution_fee_per_gallon = 0.105 # as defined in the problem statement
sample_size = 1000 # as defined in the problem statement

# Simulating the number of occupied units for each of the 1000 days (samples) using a binomial distribution
occupied_units = np.random.binomial(condos, occupancy_rate, sample_size)

# Simulating the wastewater production for each sample
wastewater_production = np.random.normal(mean_wastewater, standard_deviation_wastewater, (sample_size, condos))
wastewater_production[wastewater_production < 0] = 0  # Ensuring no negative wastewater values as negative wastewater is not realistic

# Total wastewater production for each sample. This code multiplies the generated wastewater by a binary matrix where each entry has a probability = occupancy rate
total_wastewater = np.sum(wastewater_production * (np.random.rand(sample_size, condos) < occupancy_rate), axis=1)

# Costs for different number of treatment devices
device_choices = [4, 5, 6, 7]  # as specified in the problem statement
expenses = {} # dictionary to store the average cost for each number of devices

# Evaluating expenses for different Number of treatment devices
for devices in device_choices:
    total_treatment_capacity = devices * treatment_capacity # total treatment capacity for the given number of devices
    treatment_expense = devices * treatment_device_expense # total treatment expenses for leasing the devices
    excess_wastewater = np.maximum(total_wastewater - total_treatment_capacity, 0) # finding the quantity of excess wastewater for each day
    pollution_expense = excess_wastewater * pollution_fee_per_gallon # pollution expense for excess wastewater
    total_expense = treatment_expense + pollution_expense
    average_expense = np.mean(total_expense)  # average daily expense over all simulations
    expenses[devices] = average_expense

# Finding the most optimal number of devices with the lowest expenses
optimal_devices = min(expenses, key=expenses.get)
optimal_expense = expenses[optimal_devices]
print(f'The optimal number of treatment devices for this problem is {optimal_devices}.')
print(f'The corresponding lowest daily cost is ${optimal_expense:.2f}.')

The optimal number of treatment devices for this problem is 6.
The corresponding lowest daily cost is $205.85.


# **Problem 2: Heating Element Problem**

In [3]:
# To solve this simulation problem, we will import the Numpy library (numpy is short for 'Numerical Python')
import numpy as np

# Constants as specified in the problem statement
mean_castings = 4.1  # average castings received each day
capacity = 5 # capacity of the high-temperature oven
cost_replacement = 800 # cost of replacing the heating element
cost_failure = 1500 # if the heating element fails, this is the cost to replace it
revenue_per_casting = 200 # revenue for every time the casting is processed
waiting_cost_per_day = 40 # waiting time which includes loss of goodwill, storage costs
days = 60 # number of days the simulation runs
simulations = 500 # number of simulations
probability_failure = [0.01, 0.07, 0.09, 0.15, 0.25] # probability of the heating element failing on days of use (1-5) respectively

def simulation(d):
    total_profit = 0 # Total profit accumulated over all simulation runs
    queue_exceeds_10_count = 0 # Counter for the number of times the queue exceeds 10 castings

    for i in range(simulations):
        queue = 0  # number of castings waiting to be processed
        profit = 0 # profit for the current simulation run
        element_age = 0 # age of the heating element in days
        queue_exceeded_10 = False # flag to check

        for day in range(days):
            # checking if the heating element fails on the current day
            if element_age > 0 and np.random.rand() < probability_failure[element_age - 1]:
                profit -= cost_failure
                element_age = 0
            else:
            # generating the number of castings for the day
                castings = np.random.poisson(mean_castings)
                queue += castings
            # processing as many castings as the oven can handle
                processed = min(queue, capacity)
                queue -= processed
            # profit from processed castings
                profit += processed * revenue_per_casting
            # deduct waiting cost
                profit -= queue * waiting_cost_per_day
                element_age += 1 # increasing the age of the heating element
                if element_age == d:
                    profit -= cost_replacement # replacing the heating element if it reaches end of life
                    element_age = 0

            if queue > 10: # checking if the queue exceeds 10 at any time during the day
                queue_exceeded_10 = True

        if queue_exceeded_10: # if the queue exceeds 10, increment the count
            queue_exceeds_10_count += 1

        total_profit += profit # aggregated total profit
  # calculating average profit and probability of queue exceeding 10
    average_profit = total_profit / simulations
    probability_queue_exceeds_10 = queue_exceeds_10_count / simulations
    return average_profit, probability_queue_exceeds_10

# Running the simulation multiple times and average the results
def average_multiple_runs(num_runs, d_values):
    profits_accumulated = {d: 0 for d in d_values} # dictionary to aggregate profits for each d
    optimal_d_counts = {d: 0 for d in d_values} # dictionary to count the times each d is optimal
    optimal_d_profits_accumulated = 0 # aggregator for profits when using optimal d
    queue_exceed_prob_accumulated = 0 # aggregator for the probability of queue exceeding 10

    for _ in range(num_runs):
        profits = {d: simulation(d)[0] for d in d_values} # running simulations for each d and store average profits
        optimal_d = max(profits, key=profits.get) # finding the d with the highest average profit
        optimal_d_profit, queue_exceed_prob = simulation(optimal_d) # running simulation for the optimal d to get detailed results

        for d in d_values:
            profits_accumulated[d] += profits[d] # aggregating profits for each d
            if d == optimal_d:
                optimal_d_counts[d] += 1 # counting the times each d is optimal
        optimal_d_profits_accumulated += optimal_d_profit # aggregating profits for optimal d
        queue_exceed_prob_accumulated += queue_exceed_prob # aggregating queue exceedance probabilities

    avg_profits = {d: profits_accumulated[d] / num_runs for d in d_values} # calculating average profits
    most_frequent_optimal_d = max(optimal_d_counts, key=optimal_d_counts.get) # finding most frequently optimal d
    avg_optimal_d_profit = optimal_d_profits_accumulated / num_runs # calculating average profit for optimal d
    avg_queue_exceed_prob = queue_exceed_prob_accumulated / num_runs # calculating average queue exceedance probability

    return avg_profits, most_frequent_optimal_d, avg_optimal_d_profit, avg_queue_exceed_prob

# Running the averaging process
d_values = range(1, 6)
avg_profits, most_frequent_optimal_d, avg_optimal_d_profit, avg_queue_exceed_prob = average_multiple_runs(10, d_values)

# Displaying the results
print(f'Average Profits across 5 days of use: {avg_profits}')
print(f'Optimal d: {most_frequent_optimal_d}')
print(f'Average Profit (Optimal d): ${avg_optimal_d_profit:.2f}')
print(f'Probability: {avg_queue_exceed_prob*100:.2f}%')


Average Profits across 5 days of use: {1: -2261.464, 2: 21282.06, 3: 27439.243999999995, 4: 30175.308, 5: 30581.351999999995}
Optimal d: 5
Average Profit (Optimal d): $30739.78
Probability: 11.06%


# **Problem 3: Maintenance Shop Operation**

In [4]:
import numpy as np
import pandas as pd

# Parameters
mean_compressors = 8.1  # average number of compressors received per day, follows poisson distribution
daily_processing_capacity = 10 # max compressors processed per day if no failure
daily_cost_per_compressor_waiting = 80 # cost for each compressor waiting in the queue
replacement_cost = 500 # replacement cost
days_to_simulate = 100 # number of days to simulate
samples = 500 # number of simulation runs to average the results
failure_chances = [0.02, 0.04, 0.10, 0.12, 0.15, 0.20, 0.25]
n_values = [2, 3, 4, 5, 6] # different values of 'n' days after which the part is replaced to test in the simulation

# Function to simulate the operation for a given n
def simulate_operation(n):
    total_costs = []
    average_queues = []
    scheduled_replacements = []
    failures = []

    for _ in range(samples):
        queue = 0
        total_cost = 0
        total_scheduled_replacements = 0
        total_failures = 0
        days_since_last_replacement = 0

        for day in range(days_to_simulate):
            compressors_today = np.random.poisson(mean_compressors)
            if days_since_last_replacement >= n:
                # Scheduled replacement
                total_cost += replacement_cost
                total_scheduled_replacements += 1
                days_since_last_replacement = 0

            # Check for part failure
            failure_chance = failure_chances[min(days_since_last_replacement, 6)]
            if np.random.random() < failure_chance:
                # Part failure
                total_cost += replacement_cost
                total_failures += 1
                days_since_last_replacement = 0
                # Process 0 to 9 compressors
                processed_today = np.random.randint(0, 10)
            else:
                # No failure, process up to 10 compressors
                processed_today = min(daily_processing_capacity, compressors_today + queue)
                days_since_last_replacement += 1

            queue += compressors_today - processed_today
            total_cost += queue * daily_cost_per_compressor_waiting

        total_costs.append(total_cost)
        average_queues.append(queue / days_to_simulate)
        scheduled_replacements.append(total_scheduled_replacements)
        failures.append(total_failures)

    return (np.mean(total_costs), np.mean(average_queues), np.mean(scheduled_replacements), np.mean(failures))

# Perform the simulation for each value of n
results = {}
for n in n_values:
    results[n] = simulate_operation(n)

# Find the value of n with the lowest total cost
optimal_n = min(results, key=lambda x: results[x][0])
optimal_result = results[optimal_n]

# Create a DataFrame to display results
df_results = pd.DataFrame(results, index=["Total Cost", "Average Queue", "Scheduled Replacements", "Failures"]).T
print("Simulation Results:")
print(df_results)
print("\nOptimal n:", optimal_n)
print("Optimal Result (Total Cost, Average Queue, Scheduled Replacements, Failures):", optimal_result)


Simulation Results:
   Total Cost  Average Queue  Scheduled Replacements  Failures
2    35856.28        0.01266                  46.730     3.036
3    30929.00        0.01836                  28.466     5.208
4    29549.36        0.02030                  19.200     6.804
5    28110.92        0.02110                  13.496     8.170
6    29081.60        0.02554                   9.406     9.554

Optimal n: 5
Optimal Result (Total Cost, Average Queue, Scheduled Replacements, Failures): (28110.92, 0.0211, 13.496, 8.17)


# **Problem 4: Computer Support Problem**

In [5]:
import numpy as np
import pandas as pd

# Parameters
num_computers = 200
prob_hardware_problem = 0.05
prob_serious_problem = 0.30
time_minor_problem = 0.75
time_serious_problem_extra = 2
cst_hours_per_day = 6
cst_cost_per_day = 300
outside_firm_cost_per_hour = 95
num_simulations = 1000

# Function to simulate one day
def simulate_day():
    num_problems = np.sum(np.random.rand(num_computers) < prob_hardware_problem)
    minor_problems = num_problems * (1 - prob_serious_problem)
    serious_problems = num_problems * prob_serious_problem

    total_hours_needed = (minor_problems * time_minor_problem) + (serious_problems * (time_minor_problem + time_serious_problem_extra))
    return total_hours_needed

# Simulation for each CST option
results = []
for num_csts in range(1, 5):
    daily_costs = []
    outside_hours = []

    for _ in range(num_simulations):
        total_hours_needed = simulate_day()
        cst_hours_available = num_csts * cst_hours_per_day
        if total_hours_needed > cst_hours_available:
            outside_hours_needed = total_hours_needed - cst_hours_available
            cost = (num_csts * cst_cost_per_day) + (outside_hours_needed * outside_firm_cost_per_hour)
            outside_hours.append(outside_hours_needed)
        else:
            cost = num_csts * cst_cost_per_day
            outside_hours.append(0)

        daily_costs.append(cost)

    avg_cost = np.mean(daily_costs)
    avg_outside_hours = np.mean(outside_hours)
    results.append((num_csts, avg_cost, avg_outside_hours))

# Create DataFrame
results_df = pd.DataFrame(results, columns=['CSTs', 'Average Cost per Day', 'Average Outside Hours per Day'])

# Display the results
results_df


,CSTs,Average Cost per Day,Average Outside Hours per Day
0,1,1019.1975,7.5705
1,2,845.8980,2.5884
2,3,930.0105,0.3159
3,4,1201.3965,0.0147
